<a href="https://colab.research.google.com/github/ErnestoAvedillo/42Barcelona/blob/main/Reto_AAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [149]:
#!pip install --upgrade gspread

In [150]:
#!pip install --upgrade pandas

In [151]:
import pandas as pd
import numpy as np
from google.colab import auth
import gspread
from google.auth import default#autenticating to google

In [152]:
gspread.__version__

'3.4.2'

In [153]:
pd.__version__

'1.5.3'

In [154]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Reading Excel AAM data.gsheet


In [155]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
file_path = "/content/drive/MyDrive/Colab Notebooks/AAM data.gsheet"
try:
    AAM_workbook = gc.open_by_key('1vR38j6NLoZ8e_B8zdlVAHdtYq5INmhb9xcRPm23llso')
    #worksheet = gc.open(file_path)
    print("Google Sheet opened successfully.")
except gspread.exceptions.SpreadsheetNotFound:
    print("Error: Spreadsheet not found.") #Convert to a DataFrame


Google Sheet opened successfully.


# **Primera parte**

Modelado de la producción

Cálculo del coste de las demandas internas.

known_references = nḿero de referencias (n).

REF = diccionario con las referencias de la empresa (TODAS , las de compra , internas  y de producto acabado.)

DC = Demanda de cliente

BOM = es una tabla de n x n con el BOM de los producto (Todos) Build of Materials

##Obteniendo un listado de las referencias.


In [156]:
BOM_worksheet = AAM_workbook.worksheet('BOM')#Convert to a DataFrame
rows = BOM_worksheet.get_all_values()#Convert to a DataFrame
df = pd.DataFrame(rows)
#creating columns name
df.columns = df.iloc[0]
df = df.iloc[1:]
REF_df = pd.concat([df['Ref Entrada'],df['Ref Salida']]).drop_duplicates()
#REF = REF_df[:].to_dict(orient='dict') does not work will try again
REF = {}
n = 0
for i in REF_df:
  REF[i] = n
  n += 1
print ("Diccionario de referencias con sus índices")
print(REF)

Diccionario de referencias con sus índices
{'AAA': 0, 'AMA': 1, 'BAJ': 2, 'AAB': 3, 'CAT': 4, 'AAE': 5, 'BAC': 6, 'CAR': 7, 'AAF': 8, 'BAD': 9, 'CAV': 10, 'AAC': 11, 'CAB': 12, 'CAX': 13, 'CAH': 14}


In [157]:
# Obtengo el numero de referencias
known_references = REF_df.shape[0]
print("Número total de referencias ", known_references)

Número total de referencias  15


## Creando el BOM que aparece en el Excel

In [158]:
#BILL OF MATERIALS
BOM = np.zeros((known_references,known_references))
for index, row in df.iterrows():
  BOM[REF[row["Ref Salida"]],REF[row["Ref Entrada"]]] = float(row["Cantidad"].replace(',', '.'))
BOM_df = pd.DataFrame(BOM,index=REF_df, columns=REF_df)
print (BOM_df)

     AAA   AMA  BAJ  AAB  CAT  AAE  BAC  CAR  AAF  BAD  CAV  AAC  CAB  CAX  \
AAA  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AMA  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
BAJ  1.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AAB  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
CAT  0.0  0.07  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AAE  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
BAC  0.0  0.00  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
CAR  0.0  0.08  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
AAF  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
BAD  0.0  0.00  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
CAV  0.0  0.05  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
AAC  0.0  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

## Obtención de la demanda de cliente

In [159]:
DC_worksheet = AAM_workbook.worksheet('DC') #Convert to a DataFrame
rows = DC_worksheet.get_all_values()#Convert to a DataFrame
DC_df = pd.DataFrame(rows)
DC_df.iloc[0][0] = None
# Set row 1 as column names
DC_df.columns = DC_df.iloc[0]

# Set column 1 as row names
DC_df = DC_df.set_index(DC_df.iloc[:, 0])

# Remove row 1 and column 1
DC_df = DC_df.iloc[1:, 1:]
print(DC_df)
#Demandas de cliente
Dias = DC_df.shape[1]
Profundidad = 10
DC = np.zeros((known_references,Profundidad,Dias))
for index, row in DC_df.iterrows():
  n = 0
  for i in row:
    DC[REF[index],0,n] = i
    n += 1
print('\n')
#print (DC)

0   SEM1 SEM2 SEM3 SEM4
CAB   30   50   40   20
CAX  100  110  110  130
CAH  300  350  310  340




Simulacion de un BOM con dos productos diferentes

**Listado de Maquinaria**

mumMaquinas = número de máquinas

DICMAQ = Diccionario de maquinaria (m x 1)

BOMMAQ = Listado de maquinaria por referencia (m x n)

##Descargamos los datos de las máquinas

In [160]:
Maquinas_worksheet = AAM_workbook.worksheet('MAQUINARIA') #Convert to a DataFrame
rows = Maquinas_worksheet.get_all_values()#Convert to a DataFrame
MAQUINAS_df = pd.DataFrame(rows)
# Set row 1 as column names
MAQUINAS_df.columns = MAQUINAS_df.iloc[0]
# Remove row 1 and column 1
MAQUINAS_df = MAQUINAS_df.iloc[1:,:]
print(MAQUINAS_df)


0 COD. MAQUINA POTENCIA
1          ENC        0
2         MEC1      125
3         MEC2      100
4         VUL1      150
5         VUL2      200


##Creamos el diccionario de las máquinas

In [161]:
DICMAQ = {"Compras":0}
MAQ_df = MAQUINAS_df['COD. MAQUINA'].drop_duplicates().sort_values()
print ("dataframe de Maquinas")
print(MAQ_df)
n = 1
for i in MAQ_df:
  DICMAQ[i] = n
  n += 1
print('\n')
print("Diccionario de máquinas")
print(DICMAQ)

numMaquinas = MAQ_df.shape[0] + 1
print ("Número de máquinas" , numMaquinas)



dataframe de Maquinas
1     ENC
2    MEC1
3    MEC2
4    VUL1
5    VUL2
Name: COD. MAQUINA, dtype: object


Diccionario de máquinas
{'Compras': 0, 'ENC': 1, 'MEC1': 2, 'MEC2': 3, 'VUL1': 4, 'VUL2': 5}
Número de máquinas 6


##Descargamos los la potencia de cada máquina

In [162]:
CPOT = np.zeros((numMaquinas,1))
for index, row in MAQUINAS_df.iterrows():
  CPOT[DICMAQ[row["COD. MAQUINA"]]] = row["POTENCIA"]
print (CPOT)#Potencia de cada Máquina

[[  0.]
 [  0.]
 [125.]
 [100.]
 [150.]
 [200.]]


## Descargamos los datos de los procesos del EXCEL

In [163]:
Procesos_worksheet = AAM_workbook.worksheet('PROCESOS') #Convert to a DataFrame
rows = Procesos_worksheet.get_all_values()#Convert to a DataFrame
Procesos_df = pd.DataFrame(rows)
# Set row 1 as column names
Procesos_df.columns = Procesos_df.iloc[0]
# Remove row 1 and column 1
Procesos_df = Procesos_df.iloc[1:,:]
print(Procesos_df)


0 Ref Salida PROCESO TIEMPO CICLO PIEZAS CAJA NUMERO OPERARIOS POTENCIA    \
1        BAJ    MEC2          100          16                1      100     
2        CAT     ENC           20          16                2        0     
3        CAB    VUL1           60          32                2      150     
4        BAC    MEC2           80          20                1      100     
5        CAR     ENC           16          20                1        0     
6        CAX    VUL2           50          60                2      200     
7        BAD    MEC1           90          18                1      125     
8        CAV     ENC           25          18                1        0     
9        CAH    VUL2           75          36                2      200     

0 Añadir piezas de compra  
1                          
2                          
3                          
4                          
5                          
6                          
7                          
8    

## Creamos el BOM de la maquinaria (Referencia vs Maquina)

In [164]:
BOMMAQ = np.zeros((numMaquinas,known_references))
for index, row in Procesos_df.iterrows():
  BOMMAQ[DICMAQ[row["PROCESO"]], REF[row["Ref Salida"]]] = 1
print (BOMMAQ)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]


##Descargamos el tiempo de ciclo de cada operacion

In [165]:
#Tiempo de ciclo estimado en los procesos
T= np.zeros((known_references, 1))
for index, row in Procesos_df.iterrows():
  T[REF[row["Ref Salida"]]] = row["TIEMPO CICLO"]
print(T)

[[  0.]
 [  0.]
 [100.]
 [  0.]
 [ 20.]
 [  0.]
 [ 80.]
 [ 16.]
 [  0.]
 [ 90.]
 [ 25.]
 [  0.]
 [ 60.]
 [ 50.]
 [ 75.]]


##Buscamos el numero de piezas que se introducirán en el mpaquetado de cada referencia

In [166]:
#Estimo las piezs por caja de cada componente
BOX = np.ones((known_references,1))
for index, row in Procesos_df.iterrows():
  BOX[REF[row["Ref Salida"]]] = row["PIEZAS CAJA"]
print (BOX)

[[ 1.]
 [ 1.]
 [16.]
 [ 1.]
 [16.]
 [ 1.]
 [20.]
 [20.]
 [ 1.]
 [18.]
 [18.]
 [ 1.]
 [32.]
 [60.]
 [36.]]


##Buscamos el numero de operarios para generar cada referencia interna

In [167]:
#Numero de operarios estimado en los procesos
NOP = np.ones((known_references,1))
for index, row in Procesos_df.iterrows():
  NOP[REF[row["Ref Salida"]]] = row["NUMERO OPERARIOS"]
print (NOP)

[[1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [2.]
 [2.]]


##Calculamos las demandas internas

In [168]:
#Voy incremetnando la matriz de demandas a medida que aumenta el nicel de fabricación
for dayIndex in range(Dias):
  for profIndex in range(Profundidad-1):
    DC[:,profIndex+1,dayIndex] = np.dot(BOM.T,DC[:,profIndex,dayIndex])
for dayIndex in range(Dias):
  print("Demandas para el día", dayIndex, '\n')
  print(DC[:,:,dayIndex],'\n')

Demandas para el día 0 

[[  0.    0.    0.   60.    0.    0.    0.    0.    0.    0. ]
 [  0.    0.   27.2   0.    0.    0.    0.    0.    0.    0. ]
 [  0.    0.   60.    0.    0.    0.    0.    0.    0.    0. ]
 [  0.   95.    0.    0.    0.    0.    0.    0.    0.    0. ]
 [  0.   60.    0.    0.    0.    0.    0.    0.    0.    0. ]
 [  0.    0.    0.  400.    0.    0.    0.    0.    0.    0. ]
 [  0.    0.  100.    0.    0.    0.    0.    0.    0.    0. ]
 [  0.  100.    0.    0.    0.    0.    0.    0.    0.    0. ]
 [  0.  100.    0.    0.    0.    0.    0.    0.    0.    0. ]
 [  0.    0.  300.    0.    0.    0.    0.    0.    0.    0. ]
 [  0.  300.    0.    0.    0.    0.    0.    0.    0.    0. ]
 [  0.  270.    0.    0.    0.    0.    0.    0.    0.    0. ]
 [ 30.    0.    0.    0.    0.    0.    0.    0.    0.    0. ]
 [100.    0.    0.    0.    0.    0.    0.    0.    0.    0. ]
 [300.    0.    0.    0.    0.    0.    0.    0.    0.    0. ]] 

Demandas para el día 1 

[[

In [169]:
#DINT se corresponde a las demandas internas
for i in list(REF):
  print("Referencia " ,i, " Demanda CLIENTE --> ", DC[REF[i],0,:])
for j in range(1, DC.shape[1] - 1):
  for i in list(REF):
    print("Referencia " ,i, " Demanda interna nivel ", j, " --> ", DC[REF[i],j,:])
for i in list(REF):
  print("Referencia " ,i, " Demanda interna (compras) nivel 0 --> ", DC[REF[i],DC.shape[1] - 1,:])


Referencia  AAA  Demanda CLIENTE -->  [0. 0. 0. 0.]
Referencia  AMA  Demanda CLIENTE -->  [0. 0. 0. 0.]
Referencia  BAJ  Demanda CLIENTE -->  [0. 0. 0. 0.]
Referencia  AAB  Demanda CLIENTE -->  [0. 0. 0. 0.]
Referencia  CAT  Demanda CLIENTE -->  [0. 0. 0. 0.]
Referencia  AAE  Demanda CLIENTE -->  [0. 0. 0. 0.]
Referencia  BAC  Demanda CLIENTE -->  [0. 0. 0. 0.]
Referencia  CAR  Demanda CLIENTE -->  [0. 0. 0. 0.]
Referencia  AAF  Demanda CLIENTE -->  [0. 0. 0. 0.]
Referencia  BAD  Demanda CLIENTE -->  [0. 0. 0. 0.]
Referencia  CAV  Demanda CLIENTE -->  [0. 0. 0. 0.]
Referencia  AAC  Demanda CLIENTE -->  [0. 0. 0. 0.]
Referencia  CAB  Demanda CLIENTE -->  [30. 50. 40. 20.]
Referencia  CAX  Demanda CLIENTE -->  [100. 110. 110. 130.]
Referencia  CAH  Demanda CLIENTE -->  [300. 350. 310. 340.]
Referencia  AAA  Demanda interna nivel  1  -->  [0. 0. 0. 0.]
Referencia  AMA  Demanda interna nivel  1  -->  [0. 0. 0. 0.]
Referencia  BAJ  Demanda interna nivel  1  -->  [0. 0. 0. 0.]
Referencia  AA

Ahora vamos a buscar el minimo común múltiplo entre las demandas y las piezas por caja
Posteriormente calcularemos el tiempo de trabajo necesario para cada un o de los niveles de producción.

T =  Tiempos de ciclo (n x 1) (Dato a recibir de cliente)

BOX = Piezas por caja (n x 1)

In [170]:
#Busco en número de cajas necesario para la produccion
DCBOX = np.copy(DC);
for dayIndex in range(Dias):
  mask = np.mod(DC[:,:,dayIndex], BOX) > 0
  ones_array = np.zeros_like(np.mod(DC[:,:,dayIndex], BOX))
  ones_array [mask] = 1
  DCBOX[:,:,dayIndex] = np.floor_divide(DC[:,:,dayIndex], BOX) +  ones_array
  print (DCBOX[:,:,dayIndex],'\n')


[[  0.   0.   0.  60.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  28.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   4.   0.   0.   0.   0.   0.   0.   0.]
 [  0.  95.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   4.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0. 400.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   5.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   5.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0. 100.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  17.   0.   0.   0.   0.   0.   0.   0.]
 [  0.  17.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0. 270.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  2.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  9.   0.   0.   0.   0.   0.   0.   0.   0.   0.]] 

[[  0.   0.   0. 100.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  34.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   7.   0.   0.   0.   0.   0.   0.   0.]
 [  0. 113.   0.   0.   0.   0.   0.   0.  

In [171]:
DCHORAS = np.copy(DC);
for dayIndex in range(Dias):
  DCHORAS[:,:,dayIndex] = np.multiply(np.multiply(DCBOX[:,:,dayIndex],T),BOX)/3600
  print(DCHORAS[:,:,dayIndex],'\n')

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         1.77777778 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.35555556 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         2.22222222 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.44444444 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         7.65       0.


MAQ = is a vector of ONES where there is an operation


In [172]:
MAQ = np.zeros((numMaquinas,Dias));
for dayIndex in range(Dias):
  MAQ[:,dayIndex] = np.dot(DCHORAS[:,:,dayIndex].sum(axis = 1), BOMMAQ.T)
  print(MAQ[:,dayIndex],'\n')

[0.         2.925      7.65       4.         0.53333333 8.41666667] 

[0.         3.65555556 9.         5.77777778 1.06666667 9.16666667] 

[0.         3.22777778 8.1        4.88888889 1.06666667 8.41666667] 

[ 0.          3.26388889  8.55        4.44444444  0.53333333 10.        ] 



##Descargo los datos del Stock inicial

In [173]:
Stock_ini_worksheet = AAM_workbook.worksheet('STOCK INICIAL') #Convert to a DataFrame
rows = Stock_ini_worksheet.get_all_values()#Convert to a DataFrame
Stock_ini_df = pd.DataFrame(rows)
# Set row 1 as column names
Stock_ini_df.columns = Stock_ini_df.iloc[0]
# Remove row 1 and column 1
Stock_ini_df = Stock_ini_df.iloc[1:,:]
print(Stock_ini_df)


0  REFERENCIAS STOCK
1          AAA   654
2          AAB  1500
3          AAC  1532
4          AAE   200
5          AAF   564
6          AMA   150
7          BAC   600
8          BAD   111
9          BAJ  2225
10         CAB   123
11         CAH   254
12         CAR   566
13         CAT   448
14         CAV   995
15         CAX  3255


In [174]:
#Convierto el Stock en un array
Stock_ini = np.zeros((known_references,1))
for index, row in Stock_ini_df.iterrows():
  Stock_ini[REF[row["REFERENCIAS"]]] = row["STOCK"]
print (Stock_ini)

[[ 654.]
 [ 150.]
 [2225.]
 [1500.]
 [ 448.]
 [ 200.]
 [ 600.]
 [ 566.]
 [ 564.]
 [ 111.]
 [ 995.]
 [1532.]
 [ 123.]
 [3255.]
 [ 254.]]


In [175]:
# Calculo el numero de cajas para cada referencia
Cajas_stock_global = np.divide(Stock_ini , BOX)
print (Cajas_stock_global)

[[ 654.        ]
 [ 150.        ]
 [ 139.0625    ]
 [1500.        ]
 [  28.        ]
 [ 200.        ]
 [  30.        ]
 [  28.3       ]
 [ 564.        ]
 [   6.16666667]
 [  55.27777778]
 [1532.        ]
 [   3.84375   ]
 [  54.25      ]
 [   7.05555556]]


##Separo stock por almacenes

In [199]:
Sim_almacen = np.ones((known_references,3))
mask1 = np.logical_not (np.sum(BOM, axis = 1) == 0)
mask3 = np.logical_not (np.sum(BOM, axis = 0) == 0)
mask2 = np.logical_not (mask1 & mask3)
Sim_almacen = np.multiply(Sim_almacen, Cajas_stock_global)
Sim_almacen [mask1,0] = 0
Sim_almacen [mask2,1] = 0
Sim_almacen [mask3,2] = 0
print (Sim_almacen)

[[ 654.            0.            0.        ]
 [ 150.            0.            0.        ]
 [   0.          139.0625        0.        ]
 [1500.            0.            0.        ]
 [   0.           28.            0.        ]
 [ 200.            0.            0.        ]
 [   0.           30.            0.        ]
 [   0.           28.3           0.        ]
 [ 564.            0.            0.        ]
 [   0.            6.16666667    0.        ]
 [   0.           55.27777778    0.        ]
 [1532.            0.            0.        ]
 [   0.            0.            3.84375   ]
 [   0.            0.           54.25      ]
 [   0.            0.            7.05555556]]


In [228]:
#Comparo el stock de cajas con la capacidad del almacen
Capa_almac_worksheet = AAM_workbook.worksheet('CAPACIDAD ALMACEN') #Convert to a DataFrame
rows = Capa_almac_worksheet.get_all_values()#Convert to a DataFrame
Capa_almac_df = pd.DataFrame(rows)
# Set row 1 as column names
Capa_almac_df.columns = Capa_almac_df.iloc[0]
# Remove row 1 and column 1
Capa_almac_df = Capa_almac_df.iloc[1:,:]
print(Capa_almac_df)

0        TIPO CANTIDAD
1     Entrada    10000
2  Intermedio    20000
3      Salida    20000


In [229]:
Capa_alm = np.zeros((3,1))
for index, row in Capa_almac_df.iterrows():
  Capa_alm[index - 1] = row["CANTIDAD"]
print (Capa_alm)
print(np.sum(Sim_almacen,axis = 0))
Almacen_capaz = np.all(np.sum(Sim_almacen,axis = 0) < Capa_alm)
print(" El almacen es capaz", Almacen_capaz)

[[10000.]
 [20000.]
 [20000.]]
[4600.          286.80694444   65.14930556]
 El almacen es capaz True


# Definición de Bases de la Simulación
## Constraints de la simulacion

LoteMin = Lote mínimo de producción (n x 1)

LotMax = Lote máximo de producción (n x 1)

TCambio = Tiempos de cambio de referencia (n x n)

Planif =  Planificacion de la producción (Maq x 24 horas x Ref)

PrioMat = Matriz de priodidades (n x 1)

*Donde n = known_references*

In [177]:
Transiciones_worksheet = AAM_workbook.worksheet('TRANSICIONES') #Convert to a DataFrame
rows = Transiciones_worksheet.get_all_values()#Convert to a DataFrame
Transiciones_df = pd.DataFrame(rows)
# Set row 1 as column names
Transiciones_df.columns = Transiciones_df.iloc[0]
# Remove row 1 and column 1
Transiciones_df = Transiciones_df.iloc[1:,:]
print(Transiciones_df)


0  PROCESO Origen Destino Tiempo
1      ENC    CAR     BAJ    100
2      ENC    BAJ     CAR    100
3      ENC    CAX     CAR    100
4      ENC    CAR     CAX    100
5      ENC    BAJ     CAX    300
6      ENC    CAX     BAJ    200
7     MEC1    CAB     BAD   1800
8     MEC1    BAD     CAB   1800
9     MEC1    CAT     BAD   1800
10    MEC1    BAD     CAT   1800
11    MEC1    CAT     CAB   3600
12    MEC1    CAB     CAT   1000
13    MEC2    CAB     BAD   1800
14    MEC2    BAD     CAB   1800
15    MEC2    CAT     BAD   1800
16    MEC2    BAD     CAT   1800
17    MEC2    CAT     CAB   3600
18    MEC2    CAB     CAT   1000
19    VUL1    CAV     CAH   3600
20    VUL1    CAH     CAV   3600
21    VUL1    BAC     CAV   3600
22    VUL1    CAV     BAC   3600
23    VUL1    BAC     CAH   3600
24    VUL1    CAH     BAC   3600
25    VUL2    CAV     CAH   3600
26    VUL2    CAH     CAV   3600
27    VUL2    BAC     CAV   3600
28    VUL2    CAV     BAC   3600
29    VUL2    BAC     CAH   3600
30    VUL2

In [178]:
Transiciones = np.zeros((numMaquinas,known_references, known_references))
for index, row in Transiciones_df.iterrows():
  Transiciones[DICMAQ[row["PROCESO"]], REF[row["Origen"]], REF[row["Destino"]]] =  row["Tiempo"]
for key in DICMAQ.keys():
  print ("Tiempos de transición para la máquina ", key, '\n')
  print (Transiciones[DICMAQ[key],:,:])

Tiempos de transición para la máquina  Compras 

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Tiempos de transición para la máquina  ENC 

[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]
 [  0.   0.   0.   0.   0

In [179]:
print (MAQ)
print (DCHORAS[:,:,0])

[[ 0.          0.          0.          0.        ]
 [ 2.925       3.65555556  3.22777778  3.26388889]
 [ 7.65        9.          8.1         8.55      ]
 [ 4.          5.77777778  4.88888889  4.44444444]
 [ 0.53333333  1.06666667  1.06666667  0.53333333]
 [ 8.41666667  9.16666667  8.41666667 10.        ]]
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         1.77777778 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.35555556 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         2.22222222 0.         0.   

# Cálculo de costes del proceso

En esta sección calcularemos los costes del proceso.

En esencia el cálculo es el siguiente:

Coste proceso [CP] = Coste mano de obra[CMO] + Coste Máquina [CMQ]

Desarrollando un poco mas este cálculo tenemos que:

 CMO = Coste operario [MO] / Tiempo de ciclo [T] / 3600

CMQ = Coste Máquina fijo [CMQF] Coste Máquina Variable  [CMQV]

CMQF = Coste fijo Máquina [CF] * T / 3600

CMQV = Coste potencia [CPOT] * Potencia Máquina [POT] * T / 3600

Sustituyento tenemos:

CP = (MO + CF + CPOT * POT) * T / 3600

Ahora nos centraremso en el cálculo de la mano de obra

MAQREF = Vector Maquinas (n x 1)

MOM = Coste de la mano de obra Turno de mañana

MOT = Coste de la mano de obra Turno de tarde

MON = Coste de la mano de obra Turno de noche

MHORARIA = MATRIZ Horaria (n x 24)

NOP =  Número de operarios (n x 1) (Dato a recibir de cliente)

CMOTHORARIO = Coste de mano de obra total (nx24) resultado de MOTURNOS * MOP *T / 3600

In [180]:
MAQREF = np.zeros((known_references,1))
mask = BOMMAQ.sum(axis = 0) > 0
MAQREF[mask] = 1
print(MAQREF)
MHORARIA = np.ones((known_references,24))
MOTURNOS = np.multiply(MHORARIA, MAQREF)
print(MOTURNOS)

[[0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1

In [181]:
MOM = 12
MOT = 14
MON = 18
MOTURNOS [:,0:6] *=  MON
MOTURNOS [:,6:14] *=  MOM
MOTURNOS [:,14:22] *=  MOT
MOTURNOS [:,22:24] *=  MON
print(MOTURNOS)

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [18. 18. 18. 18. 18. 18. 12. 12. 12. 12. 12. 12. 12. 12. 14. 14. 14. 14.
  14. 14. 14. 14. 18. 18.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [18. 18. 18. 18. 18. 18. 12. 12. 12. 12. 12. 12. 12. 12. 14. 14. 14. 14.
  14. 14. 14. 14. 18. 18.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [18. 18. 18. 18. 18. 18. 12. 12. 12. 12. 12. 12. 12. 12. 14. 14. 14. 14.
  14. 14. 14. 14. 18. 18.]
 [18. 18. 18. 18. 18. 18. 12. 12. 12. 12. 12. 12. 12. 12. 14. 14. 14. 14.
  14. 14. 14. 14. 18. 18.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [18. 18. 18. 18. 18. 18. 12. 12. 12. 12. 12. 12. 12. 12. 14. 14. 14. 14.
  14. 14. 14. 14.

In [182]:
CMOTHORARIO  =np.multiply(MOTURNOS, np.multiply(T,NOP)) / 3600
print(CMOTHORARIO)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.5        0.5        0.5        0.5        0.5        0.5
  0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
  0.33333333 0.33333333 0.38888889 0.38888889 0.38888889 0.38888889
  0.38888889 0.38888889 0.38888889 0.38888889 0.5        0.5       ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        


Ahora nos centraremso en el cálculo de la mano de obra

MAQ = Vector Maquinas (n x 1)

CMF = Coste máquina fijo (n x 24)

CPOT = Tabla de coste de potencias (n x 24)

POT = Potencia de Maquinaria (n x 1) KWH

CELECM = Coste potencia turno mañana

CELECT = coste potencia turno tarde

CELECN = Coste potencia turno Noche

CMVTURNOS = Coste maquina variable (n x 24))

T =  Tiempos de ciclo (n x 1) (Definidos anteriormente)

CMQVTHORARIO = Coste de maquina total (nx24) resultado de CMVTURNOS * POT *T / 3600

CMTOT = (CMF + CMQVTHORARIO) * T / 3600

In [183]:
from genericpath import commonprefix
#Coste maquina definido por cliente CMF
cmfijo = 5  #5 es un valor estimado
CMF = np.ones((known_references,24))
CMF = np.multiply(CMF, MAQREF)
CMF = CMF * cmfijo
print (CMF)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [5. 5. 5. 5. 5. 5. 5. 5.

In [184]:
CMTURNOS = np.ones((known_references,24))
CMTURNOS = np.multiply(CMTURNOS, MAQREF)
print(CMTURNOS)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.

In [185]:

CELECM = 0.12
CELECT = 0.14
CELECN = 0.16
CMTURNOS [:,0:6] *=  CELECN
CMTURNOS [:,6:14] *=  CELECM
CMTURNOS [:,14:22] *=  CELECT
CMTURNOS [:,22:24] *=  CELECN
print(CMTURNOS)

[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.16 0.16 0.16 0.16 0.16 0.16 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12
  0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.16 0.16]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.16 0.16 0.16 0.16 0.16 0.16 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12
  0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.16 0.16]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.16 0.16 0.16 0.16 0.16 0.16 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12
  0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.16 0.16]
 [0.16 0.16 0.16 0.16 0.16 0.16 0.12 0.12 0.12 0.12 0.12 0.12 0.12 0.12
  0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.16 0.16]
 [0.   0

In [186]:
POT_referencia = np.multiply(BOMMAQ,CPOT)
POT_referencia = np.expand_dims(POT_referencia.sum(axis = 0), axis = 0)

print(POT_referencia)

[[  0.   0. 100.   0.   0.   0. 100.   0.   0. 125.   0.   0. 150. 200.
  200.]]


In [187]:
CMQVTHORARIO  = np.multiply(CMTURNOS , POT_referencia.T)
print(CMQVTHORARIO)

[[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [16.  16.  16.  16.  16.  16.  12.  12.  12.  12.  12.  12.  12.  12.
  14.  14.  14.  14.  14.  14.  14.  14.  16.  16. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [16.  16.  16.  16.  16.  16.  12.  12.  12.  12.  12.  12.  12.  12.
  14.  14.  14.  14.  14.  14.  14.  14.  16.  16. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   

In [188]:
CMTOT = np.multiply((CMF + CMQVTHORARIO), T) / 3600
print (CMTOT)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.58333333 0.58333333 0.58333333 0.58333333 0.58333333 0.58333333
  0.47222222 0.47222222 0.47222222 0.47222222 0.47222222 0.47222222
  0.47222222 0.47222222 0.52777778 0.52777778 0.52777778 0.52777778
  0.52777778 0.52777778 0.52777778 0.52777778 0.58333333 0.58333333]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0. 

El coste total es el resultado de sumar el coste máquina y el de operario
CTOTHORARIO

In [189]:
CTOTHORARIO = CMOTHORARIO + CMTOT
print (CTOTHORARIO)

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [1.08333333 1.08333333 1.08333333 1.08333333 1.08333333 1.08333333
  0.80555556 0.80555556 0.80555556 0.80555556 0.80555556 0.80555556
  0.80555556 0.80555556 0.91666667 0.91666667 0.91666667 0.91666667
  0.91666667 0.91666667 0.91666667 0.91666667 1.08333333 1.08333333]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0. 

# **Inicio de modelado de simulación**
El modelo de la simulación requiere que se definan:
1. Descipción de los componentes del sistema (ver zona anterior)
2. El reloj de la simulación (arreglo de tiempos) se considera trabajar por semanas de 6 días 0-lunes -- 5 sábado
3. Definición del estado inicial del sistema
4. El modelo para el cambio de estado
5. Procedimiento para adelantar el reloj
Como resultado del sistema se provee:
1. Condición inicial
2. Listado de eventos (aleatorios + paramétricos)
3. Resultado obtenido de la ejecución en parámetros evaluables de la efectividad del resultado.

*Reloj de la simulación* - se establece como una matriz horarira de 24 x 6 días (programación semanal)

In [190]:
Calendario_Sim = np.zeros((24,6))
print(Calendario_Sim)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


*Definir el estado inicial* - Estado de las máquinas y almacenamiento

Se definie un Coste de referencia para el calculo del coste de material en el almacén

In [191]:
Process_status = np.zeros((len(DICMAQ),1))
Process_status[DICMAQ["Inyeccion"]] = REF["5"]
Process_status[DICMAQ["Mecanizado"]] = REF["8"]
STOCK = {"Referencia":0,"Volumen":1,"Coste":2}
Reference_cost = np.zeros((len(REF),1))
Reference_cost[REF["1"]] = 12.8
Reference_cost[REF["2"]] = 23.7
Reference_cost[REF["3"]] = 18.4
Reference_cost[REF["4"]] = 6.2
Reference_cost[REF["5"]] = 50.1
Reference_cost[REF["6"]] = 80.2
Reference_cost[REF["7"]] = 67.4
Reference_cost[REF["8"]] = 73.9

Almacen_MP = np.zeros((3,known_references))
Almacen_MP[STOCK["Referencia"],REF["1"]] = REF["1"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["1"]] = 5
Almacen_MP[STOCK["Coste"],REF["1"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["1"]]

Almacen_MP[STOCK["Referencia"],REF["2"]] = REF["2"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["2"]] = 7
Almacen_MP[STOCK["Coste"],REF["2"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["2"]]

Almacen_MP[STOCK["Referencia"],REF["3"]] = REF["3"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["3"]] = 1
Almacen_MP[STOCK["Coste"],REF["3"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["3"]]

Almacen_MP[STOCK["Referencia"],REF["4"]] = REF["4"] # Aunque sea redundante, es solo un tema de legibilidad
Almacen_MP[STOCK["Volumen"],REF["4"]] = 2
Almacen_MP[STOCK["Coste"],REF["4"]] = Almacen_MP[STOCK["Volumen"],0] * Reference_cost[REF["4"]]


# Notas Alf - Borrar
#MAQ_TRANS_TIEMPO[2,TRANS["Maquina"]] = DICMAQ["Inyeccion"] / "Mecanizado"
#MAQ_TRANS_TIEMPO[2,TRANS["Origen"]] = REF["5"]             / REF["7"]
#MAQ_TRANS_TIEMPO[2,TRANS["Destino"]] = REF["6"]            / REF["8"]

print("Reference costs:",Reference_cost,"\n")
print("Current status of RowMat Stock\n",Almacen_MP)
print("Process Current State: ",Process_status)

KeyError: ignored